In [ ]:
from db_import import *
from datetime import datetime, timedelta
import numpy as np

pd.set_option('display.max_columns', 30)
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# Data Import
end_date = datetime.now().strftime('%Y-%m-%d')
query_obj = Queries('2022-01-01', end_date)
db_obj = DBImport(db_type='cscart')

analytics = db_obj.data_import(query_obj.analytics_query)
brand = db_obj.data_import(query_obj.brand_query)
inventory = db_obj.data_import(query_obj.inventory_query)
category = db_obj.data_import(query_obj.category_query)
mainexposure = db_obj.data_import(query_obj.mainexposure_query)

In [ ]:
# basic preprocessing
analytics['purchased_ymd'] = pd.to_datetime(analytics.purchased_at).dt.normalize() # 시간 제외한 날짜만
brand['brand'] = brand['brand'].apply(lambda x: x.strip())

# 재고테이블 product_id와 barcode 매칭
prod_bar_match = analytics[['product_id', 'barcode']].drop_duplicates()
inventory = pd.merge(inventory, prod_bar_match, on='barcode', how='left')
inventory = inventory[inventory['product_id'].notnull()]
inventory['product_id'] = inventory['product_id'].astype('int')

# 판매 관련 확인

- 판매 급상승의 경우, 과거 판매가 된 날짜만을 바탕으로 진행. 즉, 재고가 있지만 판매가 일어나지 않아 **판매량을 0으로 집계하는 과정은 생략 (∵ ① 지나친 저회전 상품에 대해 급상승을 보고싶은 것도 아니고, ② 재고 정보가 부정확할 수도 있음. 무엇보다도, ③ 0을 복구해줌으로서 분모에 위치시킬 경우 왜곡이 발생할 수 있기 때문)**
- 현재 날짜를 기준으로 최근 일주일을 **타겟기간**, 일주일전으로부터 14일 전까지를 **비교기간**으로 설정 → 각 기간 내에서도 OOS로 인해 판매가 일어나지 않았을 경우, 단순 합으로 비교가 어렵기 때문에 주단위 판매량 합이 아니라, **각 기간의 일평균**으로 연산
- 위와같이 설정할 경우, 비교기간동안 판매가 0건인 상품의 경우 inf로 연산되기 때문에 **비교기간동안 판매가 없는 상품**의 경우 별도 처리 필요 → **1로 대체** (∵ 최근 입고 등의 이슈로 최근 판매량이 급상승 한 상품은 주로 추적을 원하기 때문에, 판매수량이 그대로 입력되도록 분모를 1로 설정. 단, 판매량이 어느정도 되어야 재입고 급상승의 의미가 있기 때문에, **타겟기간 최근 판매량에 threshold 설정**)
- ~~상대적으로 중요도가 떨어지는, **판매량이 낮은 상품들**도 비율상으로 커져보일 수 있음. **비교기간 평균판매량에 threshold 설정**~~ → 제외. 바로 위에서 최근 재입고 상품의 경우 분모가 1이되어야 하는데, 비교기간 평균판매량에 threshold가 1보다 크게 걸리면 해당 상품들은 제외되기 때문
- 카테고리의 경우, 기존의 category_M, category_S 두 카테고리를 활용하는 것이 아니라, 상품:카테고리의 1:N 매칭이 이루어지도록 카테고리 테이블을 활용하여 한 상품이 여러 카테고리에 중복 집계가 가능하도록 연산
- 브랜드, 카테고리별 결과의 경우 상위 10개의 결과에 해당하는 상품별 상세정보 테이블 추가로 생성

In [ ]:
# target and compare dates
target_end = datetime.now().strftime('%Y-%m-%d')
target_start = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')
compare_end = (datetime.now() - timedelta(days=8)).strftime('%Y-%m-%d')
compare_start = (datetime.now() - timedelta(days=15)).strftime('%Y-%m-%d')

In [ ]:
### aggregation

## 1. 상품별 결과
target_qty_threshold = 5 # 타겟기간 평균판매량의 threshold. 급상승에서 타겟기간 최소판매기준
compare_qty_threshold = 5 # 비교기간 평균판매량의 threshold. 급하락에서 과거기간 최소판매기준

daily_sales = analytics.groupby(['product_id', 'purchased_ymd'])['product_qty'].sum().reset_index()

target_sales = daily_sales[(daily_sales['purchased_ymd']>target_start)&(daily_sales['purchased_ymd']<=target_end)]
target_sales = target_sales.groupby('product_id').agg({'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
target_sales = target_sales.rename(columns={'product_qty':'target_qty', 'purchased_ymd':'target_days'})

compare_sales = daily_sales[(daily_sales['purchased_ymd']>compare_start)&(daily_sales['purchased_ymd']<=compare_end)]
compare_sales = compare_sales.groupby('product_id').agg({'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
compare_sales = compare_sales.rename(columns={'product_qty':'compare_qty', 'purchased_ymd':'compare_days'})

prod_sales = pd.merge(target_sales, compare_sales, on='product_id', how='outer')
non_compare = prod_sales[(prod_sales['compare_qty'].isnull())&(prod_sales['target_qty']>=target_qty_threshold)] # 비교기간 판매는 없었지만, 재입고 등의 이슈로 인해 급상승한 상품을 살려주기 위한 작업
prod_sales = pd.concat([prod_sales[prod_sales['compare_qty'].notnull()], non_compare])
prod_sales['target_qty'] = prod_sales['target_qty'].fillna(0)

# growth_rate 성장률 계산을 위해서는 판매량과 판매일자를 1로 채워줘야 하는데, 실제 값은 0으로 해야하기 때문에 저장해두고 추후 복구
target_days = prod_sales['target_days'].fillna(0)
compare_qty = prod_sales['compare_qty'].fillna(0)
compare_days = prod_sales['compare_days'].fillna(0)
prod_sales['target_days'] = prod_sales['target_days'].fillna(1) # target_qty는 0 그대로여도 상관없음. 항상 최우선 분자에 있기 때문
prod_sales['compare_qty'] = prod_sales['compare_qty'].fillna(1)
prod_sales['compare_days'] = prod_sales['compare_days'].fillna(1)
prod_sales['growth_rate_daily'] = (prod_sales['target_qty']/prod_sales['target_days']) / (prod_sales['compare_qty']/prod_sales['compare_days'])
prod_sales['target_days'] = target_days
prod_sales['compare_qty'] = compare_qty
prod_sales['compare_days'] = compare_days
prod_sales = prod_sales.reset_index()

# 후처리
prod_sales['compare_qty'] = prod_sales['compare_qty'].astype('int')
prod_sales['compare_days'] = prod_sales['compare_days'].astype('int')
# prod_sales = pd.merge(prod_sales, brand, on='product_id', how='left')


# 최종결과
prod_top20 = prod_sales[prod_sales['target_qty']>=target_qty_threshold].sort_values(by=['growth_rate_daily', 'target_qty'], ascending=[False, False]).head(20) # 성장률 우선정렬, 다음은 qty 기준
prod_bot20 = prod_sales[prod_sales['compare_qty']>=compare_qty_threshold].sort_values(by=['growth_rate_daily', 'compare_qty'], ascending=[True, False]).head(20)

In [ ]:
prod_top20

In [ ]:
prod_bot20

In [ ]:
class CauseTracker():
    def __init__(self, mainexpose_df, inventory_df):
        self.mainexpose_df = mainexpose_df
        self.inventory_df = inventory_df
        
#     def expose_type(self, product_id):
#         if product_id in self.newly_expose:
#             return '메인노출'
#         elif product_id in self.newly_drop:
#             return '노출제외'
#         elif product_id in self.always_expose:
#             return '항상노출'
#         else:
#             return '항상노출안함'
    
    def check_mainexpose(self, prod_id):
        target_expose = self.mainexpose_df[(self.mainexpose_df['date']>target_start)&(self.mainexpose_df['date']<=target_end)]
        compare_expose = self.mainexpose_df[(self.mainexpose_df['date']>compare_start)&(self.mainexpose_df['date']<=compare_end)]
        self.newly_expose = list(set(target_expose['product_id']) - set(compare_expose['product_id'])) # 타겟기간에 새로 노출된 상품들
        self.newly_drop = list(set(compare_expose['product_id']) - set(target_expose['product_id'])) # 비교기간에 노출되었다가 메인에서 사라진 상품들
        self.always_expose = list(set(compare_expose['product_id']) & set(target_expose['product_id'])) # 항상 노출된 상품들 
        if prod_id in self.newly_expose:
            return '메인노출'
        elif prod_id in self.newly_drop:
            return '노출제외'
        elif prod_id in self.always_expose:
            return '항상노출'
        else:
            return '항상노출안함'

In [ ]:
tracker = CauseTracker(mainexposure, inventory)
prod_bot20['product_id'].apply(lambda x: tracker.check_mainexpose(x))

### 메인노출기준 원인파악

In [ ]:
target_expose = mainexposure[(mainexposure['date']>target_start)&(mainexposure['date']<=target_end)]
target_expose = target_expose.sort_values(by='date', ascending=False).drop_duplicates(subset='product_id')
target_expose = target_expose.rename(columns={'main_rec_category': 'target_category'})
compare_expose = mainexposure[(mainexposure['date']>compare_start)&(mainexposure['date']<=compare_end)]
compare_expose = compare_expose.sort_values(by='date', ascending=False).drop_duplicates(subset='product_id')
compare_expose = compare_expose.rename(columns={'main_rec_category': 'compare_category'})

prod_expose = pd.merge(target_expose[['product_id', 'target_category']], compare_expose[['product_id', 'compare_category']], on='product_id', how='outer')

In [ ]:
pd.merge(prod_top20, prod_expose, on='product_id', how='left')

In [ ]:
pd.merge(prod_bot20, prod_expose, on='product_id', how='left')

### 재고기준 원인파악

In [ ]:
prod_bar_match = inventory[['product_id', 'barcode']].drop_duplicates()

target_inventory = inventory[(inventory['sys_time']>target_start)&(inventory['sys_time']<=target_end)]
# target_inventory = target_inventory[target_inventory['sys_time']==target_inventory['sys_time'].max()] # 재고는 일단위 갱신정보이기 때문
compare_inventory = inventory[(inventory['sys_time']>compare_start)&(inventory['sys_time']<=compare_end)]
# compare_inventory = compare_inventory[compare_inventory['sys_time']==compare_inventory['sys_time'].max()]

target_inventory = target_inventory.groupby('barcode')['amount'].max().reset_index()
target_inventory = target_inventory.rename(columns={'amount': 'target_amount'})
compare_inventory = compare_inventory.groupby('barcode')['amount'].max().reset_index()
compare_inventory = compare_inventory.rename(columns={'amount': 'compare_amount'})

bar_inventory = pd.merge(target_inventory, compare_inventory, on='barcode', how='outer')
bar_inventory = bar_inventory.fillna(0)
bar_inventory = pd.merge(bar_inventory, prod_bar_match, on='barcode', how='left')

# # 급상승/급하락은 가장 인기많은 옵션의 영향을 받고, 재고정보는 이미 판매의 결과가 반영된 값이니, 재고변동량이 가장 큰 옵션 상위 2개를 기준으로 집계
# bar_inventory['abs_gap'] = np.abs(bar_inventory['target_amount'] - bar_inventory['compare_amount'])
# bar_inventory = bar_inventory.sort_values(by='abs_gap', ascending=False) # product_id별 상위 2개 옵션 고르기 위해
# bar_inventory = bar_inventory.groupby('product_id').head(1)

prod_inventory = bar_inventory.groupby('product_id')['target_amount', 'compare_amount'].sum().reset_index()
prod_inventory['amt_gap'] = prod_inventory['target_amount'] - prod_inventory['compare_amount']

prod_inventory

In [ ]:
pd.merge(prod_top20, prod_inventory, on='product_id', how='left')

In [ ]:
pd.merge(prod_bot20, prod_inventory, on='product_id', how='left')

원인미상의 케이스 확인

In [ ]:
inventory[inventory['product_id']==2281].sort_values(by='sys_time', ascending=False).head(20)

In [ ]:
analytics[analytics['product_id']==2281].sort_values(by='purchased_ymd', ascending=False).head(20)

In [ ]:
inventory[inventory['product_id']==3420].sort_values(by='sys_time', ascending=False).head(20)

In [ ]:
analytics[analytics['product_id']==3420].sort_values(by='purchased_ymd', ascending=False).head(20)

위 두 케이스는 디버깅 결과, user_id == 0 이라는 이상판매기록이 발생

In [ ]:
inventory[inventory['product_id']==6169].sort_values(by='sys_time', ascending=False).head(20)

In [ ]:
analytics[analytics['product_id']==6169].sort_values(by='purchased_ymd', ascending=False).head(20)

6169 상품의 경우에는 가장 많이 팔리는 상품의 옵션이 나타나지 않음.

In [ ]:
### aggregation

## 1. 상품별 결과
target_qty_threshold = 5 # 비교기간 판매가 없는 상품의 타겟기간 평균판매량의 threshold
compare_qty_threshold = 2 # 비교기간 평균판매량의 threshold

daily_sales = analytics.groupby(['product_id', 'purchased_ymd'])['product_qty'].sum().reset_index()

target_sales = daily_sales[(daily_sales['purchased_ymd']>target_start)&(daily_sales['purchased_ymd']<=target_end)]
target_sales = target_sales.groupby('product_id').agg({'product_qty':'mean', 'purchased_ymd': pd.Series.nunique})
target_sales = target_sales.rename(columns={'product_qty':'target_qty', 'purchased_ymd':'target_days'})

compare_sales = daily_sales[(daily_sales['purchased_ymd']>compare_start)&(daily_sales['purchased_ymd']<=compare_end)]
compare_sales = compare_sales.groupby('product_id').agg({'product_qty':'mean', 'purchased_ymd': pd.Series.nunique})
compare_sales = compare_sales.rename(columns={'product_qty':'compare_qty', 'purchased_ymd':'compare_days'})

prod_sales = pd.merge(target_sales, compare_sales, on='product_id', how='left')
non_compare = prod_sales[(prod_sales['compare_qty'].isnull())&(prod_sales['target_qty']>=target_qty_threshold)]
prod_sales = pd.concat([prod_sales[prod_sales['compare_qty'].notnull()], non_compare])
prod_sales['growth_rate_daily'] = prod_sales['target_qty'] / prod_sales['compare_qty']
prod_sales = prod_sales.sort_values(by='growth_rate_daily', ascending=False)
prod_sales = prod_sales[prod_sales['compare_qty']>compare_qty_threshold]
prod_sales = prod_sales.reset_index()

# 아래 브랜드, 카테고리의 상품별 상세를 위한 정보 추가한 테이블 생성
prod_sales_detail = pd.merge(prod_sales, brand, on='product_id', how='left')
prod_sales_detail = pd.merge(prod_sales_detail, category, on='product_id', how='left')
prod_sales_detail = pd.merge(prod_sales_detail, analytics[['product_id', 'product_name_kor']].drop_duplicates(), on='product_id', how='left')

In [ ]:
## 2. 브랜드별 결과
target_qty_threshold = 5 # 비교기간 판매가 없는 상품의 타겟기간 평균판매량의 threshold
compare_qty_threshold = 2 # 비교기간 평균판매량의 threshold

daily_sales = analytics.groupby(['product_id', 'purchased_ymd'])['product_qty'].sum().reset_index()
daily_sales = pd.merge(daily_sales, brand, on='product_id', how='left')
daily_sales = daily_sales[daily_sales['brand'].notnull()] # 브랜드별 결과이기 때문에, 브랜드 정보 없는 상품은 제외

target_sales = daily_sales[(daily_sales['purchased_ymd']>target_start)&(daily_sales['purchased_ymd']<=target_end)]
target_sales = target_sales.groupby('brand').agg({'product_qty':'mean', 'purchased_ymd': pd.Series.nunique})
target_sales = target_sales.rename(columns={'product_qty':'target_qty', 'purchased_ymd':'target_days'})

compare_sales = daily_sales[(daily_sales['purchased_ymd']>compare_start)&(daily_sales['purchased_ymd']<=compare_end)]
compare_sales = compare_sales.groupby('brand').agg({'product_qty':'mean', 'purchased_ymd': pd.Series.nunique})
compare_sales = compare_sales.rename(columns={'product_qty':'compare_qty', 'purchased_ymd':'compare_days'})

brand_sales = pd.merge(target_sales, compare_sales, on='brand', how='left')
non_compare = brand_sales[(brand_sales['compare_qty'].isnull())&(brand_sales['target_qty']>=target_qty_threshold)]
brand_sales = pd.concat([brand_sales[brand_sales['compare_qty'].notnull()], non_compare])
brand_sales['growth_rate_daily'] = brand_sales['target_qty'] / brand_sales['compare_qty']
brand_sales = brand_sales.sort_values(by='growth_rate_daily', ascending=False)
brand_sales = brand_sales[brand_sales['compare_qty']>compare_qty_threshold]
brand_sales = brand_sales.reset_index()

# 브랜드 상위10개 결과에 해당하는 상품별 상세정보
brand_sales_top10 = brand_sales.head(10)
top10_brands = brand_sales_top10['brand'].tolist()
top10_brands_detail = []
for a_brand in top10_brands: # top10의 순서 유지하기 위해 for문 활용
    brand_prod = prod_sales_detail[prod_sales_detail['brand']==a_brand]
    top10_brands_detail.append(brand_prod)
top10_brands_detail = pd.concat(top10_brands_detail, ignore_index=True)
top10_brands_detail = top10_brands_detail[['brand', 'product_id', 'product_name_kor', 'target_qty', 'compare_qty', 'growth_rate_daily']].drop_duplicates()

In [ ]:
## 3. 카테고리별 결과
target_qty_threshold = 5 # 비교기간 판매가 없는 상품의 타겟기간 평균판매량의 threshold
compare_qty_threshold = 2 # 비교기간 평균판매량의 threshold

daily_sales = analytics.groupby(['product_id', 'purchased_ymd'])['product_qty'].sum().reset_index()
daily_sales = pd.merge(daily_sales, category, on='product_id', how='left')
daily_sales = daily_sales[daily_sales['category'].notnull()] # 브랜드별 결과이기 때문에, 브랜드 정보 없는 상품은 제외

target_sales = daily_sales[(daily_sales['purchased_ymd']>target_start)&(daily_sales['purchased_ymd']<=target_end)]
target_sales = target_sales.groupby('category').agg({'product_qty':'mean', 'purchased_ymd': pd.Series.nunique})
target_sales = target_sales.rename(columns={'product_qty':'target_qty', 'purchased_ymd':'target_days'})

compare_sales = daily_sales[(daily_sales['purchased_ymd']>compare_start)&(daily_sales['purchased_ymd']<=compare_end)]
compare_sales = compare_sales.groupby('category').agg({'product_qty':'mean', 'purchased_ymd': pd.Series.nunique})
compare_sales = compare_sales.rename(columns={'product_qty':'compare_qty', 'purchased_ymd':'compare_days'})

category_sales = pd.merge(target_sales, compare_sales, on='category', how='left')
non_compare = category_sales[(category_sales['compare_qty'].isnull())&(category_sales['target_qty']>=target_qty_threshold)]
category_sales = pd.concat([category_sales[category_sales['compare_qty'].notnull()], non_compare])
category_sales['growth_rate_daily'] = category_sales['target_qty'] / category_sales['compare_qty']
category_sales = category_sales.sort_values(by='growth_rate_daily', ascending=False)
category_sales = category_sales[category_sales['compare_qty']>compare_qty_threshold]
category_sales = category_sales.reset_index()

# 카테고리 상위10개 결과에 해당하는 상품별 상세정보
category_sales_top10 = category_sales.head(10)
top10_categorys = category_sales_top10['category'].tolist()
top10_categorys_detail = []
for a_category in top10_categorys: # top10의 순서 유지하기 위해 for문 활용
    category_prod = prod_sales_detail[prod_sales_detail['category']==a_category]
    top10_categorys_detail.append(category_prod)
top10_categorys_detail = pd.concat(top10_categorys_detail, ignore_index=True)
top10_categorys_detail = top10_categorys_detail[['category', 'product_id', 'product_name_kor', 'target_qty', 'compare_qty', 'growth_rate_daily']].drop_duplicates()

- 적어도 몇 개 이상 팔린 상품들에 대해서 필터링(급하락도 마찬가지)
- 재고는 바코드, 판매는 상품단위라 확인이 쉽지 않음
    - 상품단위로 재고를 확인하면 인기옵션에 편중된 상품일 경우 결과가 희석되는 케이스가 발생하고 (ex. 인기옵션재고부족으로 인하여 판매량이 감소하였을 때, 비인기옵션은 재고가 충분하기면, 합산 시 현재 재고가 많다고 뜰 수도 있음)
    - 바코드단위로 결과를 종합하면 세세하나, 결과가 흩뿌려져 보이고, 다양한 옵션을 묶어서 보는게 좋을 경우에 대해 볼 수 없음 (ex. 카딜로벨트의 경우 S 사이즈가 인기 편중되지만, 고객마다 옵션을 골고루 사가는 가상의 상품의 경우에는 묶어서 보는 것이 좋지만 볼 수 없음)
    - → 지금은 옵션별 상위 2개의 재고변동 상품 가져옴 (재입고가 되었을 수 있기 때문에 판매량 기준 인기 옵션으로 보는게 더 정확할 수 있음)
    
- 또한, 재고는 일단위, 판매는 주단위 합산이라 비교가 쉽지 않음. 재고는 snapshot 데이터라 언제부터 언제까지의 재고 라는 것이 불가능함.
    - → 이 문제를 해결하기 위해 일단 재고를 상품별 각 구간의 max값을 가져옴